In [1]:
import random
import pandas as pd
import numpy as np
# !python3 -m pip install -U scikit-learn
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

# this function calculates all the scores - accuracy, f1, precision, recall, and returns them in a list
def get_accuracy(column_cat, model,file_cat='',lang='en',process='validation'):
    try:
        if lang=='cn':
            path = 'outputs/'+process+'/Chinese/'+model+'/'+file_cat+'.csv'
            print(path)
            df_pred = pd.read_csv(path)
            if process == 'validation':
                labels = '../data_cleaning/output/validation_cn.csv'
            else:
                labels='../data_cleaning/output/test_cn.csv'
        else:
            path = 'outputs/'+process+'/English/'+model+'/'+file_cat+'.csv'
            print(path)
            df_pred = pd.read_csv(path)
            if process == 'validation':
                labels = '../data_cleaning/output/validation_en.csv'
            else:
                labels='../data_cleaning/output/test_en.csv'
#         print('here1')
        df_true = pd.read_csv(labels)
#         print('here2')
        y_pred = df_pred[column_cat]
        y_true = df_true[column_cat]
        output = df_true[column_cat].value_counts()
#         print('here3')
        accuracy = round(100*accuracy_score(y_true, y_pred), 2)
        scores = precision_recall_fscore_support(y_true, y_pred, average='weighted')
#         print('here4')
        return [column_cat,accuracy, max(output[0], output[1])/len(y_pred), round(100*scores[0],2), round(100*scores[1],2), round(100*scores[2],2)]#overlap/len(y_pred)
    except:
        print("category",column_cat,"doesn't exist")
        return None

# all_scores_file is the name of the csv file that contains all the scores of all files
# criteria is the type of category we're looking into
# model_num is the model's version
# pred is the 1-D array with the predicted values
# prompt is the file path of the prompt file
def add_score(all_scores_file, model_num,column_cat, file_cat, prompt,lang, clear=False, process='validation'):
    try:
        df_all = pd.read_csv(all_scores_file)
#         df_f1 = pd.read_csv(f1_file)
        print("f1 score file found for",file_cat)
    except:
        df_all = pd.DataFrame(columns=['file', 'accuracy','majority_baseline','precision','recall','f1','prompt'])
#         df_f1 = pd.DataFrame(columns=['category','f1'])
        print("NO f1 score file found for",file_cat)
    try:
        # gets all the scores and store the scores into all_scores_file
        if clear:
            df_all = df_all.iloc[0:0]
        if "llama" in all_scores_file:
            model_num = "llama/"+model_num
        result_all = get_accuracy(column_cat, model_num,file_cat,lang, process)
        result_all.append(prompt)
#         else:
        df_all.loc[len(df_all.index)] = result_all
        df_all.to_csv(all_scores_file, index = False, encoding='utf-8')
        
        # gets the f1 score and store it into f1_file. The purpose of this is to make it easier to compile all the f1 data.
#         result_f1 = [result_all[0]]+[result_all[5]]
#         df_f1.loc[len(df_f1.index)] = result_f1
#         df_f1.to_csv(f1_file, index = False, encoding='utf-8')
        
    except: 
        print("category",file_cat,"doesn't exist")


file = ['name','specialty','positivity','culture','location','ambiance', 'creative','romanized']
col = ['Personal_Name','Specialty','Positivity','Culture','Location','Ambiance', 'Pun_Creative','Romanized']
# cat=['creative']
# crit = ['Pun_Creative']

# run this to get the scores
# pred and crit are the different categories we're calculating scores for. 
def score_to_file(file, model_n,model_store,col, lang='en', clear=False,model="GPT", process='validation'):

    all_dfs=[]
    for i in range(len(file)):
        file_cat = file[i]
        column_cat = col[i]
        if lang=='en':
            store = 'scores/'+process+'/'+model+'/English/'+model_n+'/'+file_cat+'_scores.csv'
            print("store:--------------------------------")
            print(store)
            prompt = 'prompt_en_'+file_cat
        elif lang=='cn':
            store = 'scores/'+process+'/'+model+'/Chinese/'+model_n+'/'+file_cat+'_scores.csv'
            prompt = 'prompt_cn_'+file_cat
            print("store:--------------------------------")
            print(store)
        add_score(store,model_n, column_cat,file_cat,prompt,lang,clear,process)
        
        try:
            df = pd.read_csv(store)
            all_dfs.append(df)
        except:
            print("df not found")
        
    if lang=='cn':
        store_all = 'scores/'+process+'/'+model+'/Chinese/'+model_n+'/'+lang+'_'+model_store+'_all_scores.csv'
    else:
        store_all = 'scores/'+process+'/'+model+'/English/'+model_n+'/'+lang+'_'+model_store+'_all_scores.csv'
        print(store_all)
    all_scores = pd.concat(all_dfs)
    all_scores.to_csv(store_all, index = False, encoding='utf-8')
        
    

#####GPT 3.5 scores calculation English
# score_to_file(file, '3.5/zero_shot/name_def','3.5_zero_shot_name_def',col, clear='True')
# score_to_file(file, '3.5/few_shot/lexicon_based','3.5_few_shot_lexicon_based',col,clear='True')
# score_to_file(file, '3.5/few_shot/rule_based','3.5_few_shot_rule_based', col,clear='True')

#####GPT 3.5 scores calculation Chinese
score_to_file(file, '3.5/zero_shot/name_def','3.5_zero_shot_name_def',col, lang='cn',clear='True')
score_to_file(file, '3.5/few_shot/lexicon_based','3.5_few_shot_lexicon_based',col,lang='cn',clear='True')
score_to_file(file, '3.5/few_shot/rule_based','3.5_few_shot_rule_based', col,lang='cn',clear='True')

##### Llama 2 scores calculation
# score_to_file(file, 'zero_shot/only_name','llama_zero_shot_only_name',col,model="llama")
# score_to_file(file, 'zero_shot/name_def','llama_zero_shot_name_def',col,model="llama")
# score_to_file(file, 'few_shot/lexicon_based','llama_few_shot_lexicon_based',col,model="llama")
# score_to_file(file, 'few_shot/rule_based','llama_few_shot_rule_based',col,model="llama")

# score_to_file(cat, '4/zero_shot/only_name','4_zero_shot_only_name',crit,'en')
# score_to_file(cat, '4/zero_shot/name_def','4_zero_shot_name_def',crit,'en')
# score_to_file(cat, '3.5/few_shot/lexicon_based','3.5_few_shot_lexicon_based',crit,'en')
# score_to_file(cat, '4/few_shot/lexicon_based','4_few_shot_lexicon_based',crit,'en')
# score_to_file(cat, '4/few_shot/rule_based','4_few_shot_rule_based',crit)

# score_to_file(cat, '3.5/zero_shot/only_name','3.5_zero_shot_only_name',crit,'cn')
# score_to_file(cat, '3.5/zero_shot/name_def','3.5_zero_shot_name_def',crit,'cn')
# score_to_file(cat, '3.5/few_shot/lexicon_based','3.5_few_shot_lexicon_based',crit,'cn')
# score_to_file(cat, '3.5/few_shot/rule_based','3.5_few_shot_rule_based',crit,'cn')

# score_to_file(cat, '4/zero_shot/only_name','4_zero_shot_only_name',crit,'cn')
# score_to_file(cat, '4/zero_shot/name_def','4_zero_shot_name_def',crit,'cn')
# score_to_file(cat, '4/few_shot/lexicon_based','4_few_shot_lexicon_based',crit,'cn')
# score_to_file(cat, '3.5/few_shot/rule_based','3.5_few_shot_rule_based',crit,'cn') * recalculated
# add_score('scores/GPT/English/creative_scores.csv','Pun_Creative','creative','training_validation/training_en.csv','prompt_en_creative_1')
# add_score(store,criteria, output, labels,prompt)
# get_accuracy(criteria,output, labels)

# score_to_file(cat, 'zero_shot/only_name','llama_zero_shot_only_name',crit,model="llama")
# score_to_file(cat, 'zero_shot/name_def','llama_zero_shot_name_def',crit,model="llama")
# score_to_file(cat, 'few_shot/lexicon_based','llama_few_shot_lexicon_based',crit,model="llama")
# score_to_file(cat, 'few_shot/rule_based','llama_few_shot_rule_based',crit,model="llama")

# print()

store:--------------------------------
scores/validation/GPT/Chinese/3.5/zero_shot/name_def/name_scores.csv
f1 score file found for name
outputs/validation/Chinese/3.5/zero_shot/name_def/name.csv
store:--------------------------------
scores/validation/GPT/Chinese/3.5/zero_shot/name_def/specialty_scores.csv
f1 score file found for specialty
outputs/validation/Chinese/3.5/zero_shot/name_def/specialty.csv
store:--------------------------------
scores/validation/GPT/Chinese/3.5/zero_shot/name_def/positivity_scores.csv
f1 score file found for positivity
outputs/validation/Chinese/3.5/zero_shot/name_def/positivity.csv
store:--------------------------------
scores/validation/GPT/Chinese/3.5/zero_shot/name_def/culture_scores.csv
f1 score file found for culture
outputs/validation/Chinese/3.5/zero_shot/name_def/culture.csv
store:--------------------------------
scores/validation/GPT/Chinese/3.5/zero_shot/name_def/location_scores.csv
f1 score file found for location
outputs/validation/Chinese/3.

/Users/nanxiliu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
# file is the name of the csv file that contains all the scores of all files
# criteria is the type of category we're looking into
# pred is the 1-D array with the predicted values
# true is the original given labels
# prompt is the file path of the prompt file
def add_score(file, criteria, pred, true, prompt):
    df = pd.read_csv(file)
    # df['prompt']=''
    result = get_accuracy(criteria, pred, true)
    # print(result)
    result.append(prompt)
    # print('===============================')
    # print(result)
    df.loc[len(df.index)] = result
    df.to_csv(file, index = False, encoding='utf-8')

add_score(store,criteria, file1, file2,prompt)
# df=pd.read_csv('./scores/positivity_scores.csv')
# print(df['f1'])
# positivity = pd.DataFrame(columns=['file', 'accuracy','majority_baseline','precision','recall','f1','prompt'])
# positivity.loc[len(positivity.index)] = result
# positivity.to_csv('./scores/positivity_scores.csv', index = False, encoding='utf-8')

Positivity
0    270
1    149
Name: count, dtype: int64
0.6443914081145584


## Distribution

In [7]:
cat = ['positivity', 'specialty', 'name', 'location', 'creative', 'ambiance','culture']
for name in cat:
    df = pd.DataFrame(columns=['file','accuracy','majority_baseline','precision','recall','f1','prompt'])
    df.to_csv('scores/Chinese/'+name+'_scores.csv',index=False,encoding='utf-8')